<a href="https://colab.research.google.com/github/0-JackFrost-0/PoseFlow/blob/main/deeppose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scipy.io as sc
import numpy as np
import glob
from os.path import basename as b
import re

joints = sc.loadmat("FLIC_Dataset/joints.mat")
joints = joints['joints'].transpose(2,0,1)
joints = joints[:, :, :2]

N_test = int(len(joints)*0.1)
perm = np.random.permutation(int(len(joints)))[:N_test].tolist()

fp_train = open('train_joints.csv', 'w')
fp_test = open('test_joints.csv', 'w')
for img_fn in sorted(glob.glob('FLIC_Dataset/images/*.jpg')):
    index = int(re.search('im([0-9]+)', b(img_fn)).groups()[0]) - 1
    str_j = [str(j) if j > 0 else '-1'
             for j in joints[index].flatten().tolist()]

    out_list = [b(img_fn)]
    out_list.extend(str_j)
    out_str = ','.join(out_list)

    if index in perm:
        print(out_str, file=fp_test)
    else:
        print(out_str, file=fp_train)
fp_train.close()
fp_test.close()

In [ ]:
import numpy as np
import pandas as pd
import cv2
import random
import matplotlib.image as mpimg
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
def model(joints):
    conv1_W = tf.Variable(tf.truncated_normal(shape=(11, 11, 3, 96)))
    conv1_b = tf.Variable(tf.zeros(96))
    conv1   = tf.nn.conv2d(joints, conv1_W, strides=[4, 4, 4, 4], padding='VALID') + conv1_b
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.local_response_normalization(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[3, 2, 2, 3], strides=[3, 2, 2, 3], padding='VALID')

    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 96, 256)))
    conv2_b = tf.Variable(tf.zeros(256))
    conv2   = tf.nn.conv2d(joints, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.local_response_normalization(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[3, 2, 2, 3], strides=[3, 2, 2, 3], padding='VALID')

    conv3_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 256, 384)))
    conv3_b = tf.Variable(tf.zeros(384))
    conv3   = tf.nn.conv2d(joints, conv3_W, strides=[1, 1, 1, 1], padding='VALID') + conv3_b
    conv3 = tf.nn.relu(conv3)

    conv4_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 384, 384)))
    conv4_b = tf.Variable(tf.zeros(384))
    conv4   = tf.nn.conv2d(joints, conv4_W, strides=[1, 1, 1, 1], padding='VALID') + conv4_b
    conv4 = tf.nn.relu(conv4)

    conv5_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 384, 256)))
    conv5_b = tf.Variable(tf.zeros(256))
    conv5   = tf.nn.conv2d(joints, conv5_W, strides=[1, 1, 1, 1], padding='VALID') + conv5_b
    conv5 = tf.nn.max_pool(conv5, ksize=[3, 2, 2, 3], strides=[3, 2, 2, 3], padding='VALID')
    conv5 = tf.nn.relu(conv5)
    conv5 = tf.nn.max_pool(conv5, ksize=[3, 2, 2, 3], strides=[3, 2, 2, 3], padding='VALID')

    conv5_inputs = int(np.prod(conv5.shape()[1:]))
    fc6_W = tf.Variable(tf.truncated_normal(shape=(conv5_inputs, 4096)))
    fc6_b = tf.Variable(tf.zeros(4096))
    fc6 = tf.matmul(joints, fc6_W) + fc6_b
    fc6 = tf.nn.relu(fc6)
    fc6 = tf.nn.dropout(fc6, 0.6)

    fc6_inputs = int(fc6.shape()[1])
    fc7_W = tf.Variable(tf.truncated_normal(shape=(fc6_inputs, 4096)))
    fc7_b = tf.Variable(tf.zeros(4096))
    fc7 = tf.matmul(fc6, fc7_W) + fc6_b
    fc7 = tf.nn.relu(fc7)
    fc7 = tf.nn.dropout(fc7, 0.6)

    fc7_inputs = int(np.prod(fc7.shape()[1]))
    fc8_W = tf.Variable(tf.truncated_normal(shape=(fc8_inputs, 14)))
    fc8_b = tf.Variable(tf.zeros(14))
    fc8 = tf.matmul(joints, fc8_W) + fc8_b

    return fc8


In [ ]:
joints = 0
# joints preprocessing still left
Epoch = 100
rate = 0.0005
logits = model(joints)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
#!/usr/bin/env python

import os
import caffe
import numpy as np
import cv2
import sys
import copy
import random

from globalvar import *

STAGE1MODEL=sys.argv[1]

#-----------------STAGE1--------------------#

def Buildnet():
    deploy='./models/deeppose/deeppose.prototxt'
    caffe_model=STAGE1MODEL
    MEAN_NPY_PATH = './models/deeppose/mean.npy'
    os.system("rm -f "+MEAN_NPY_PATH)

    print "Create mean.npy"
    MEAN_PROTO_PATH = './models/deeppose/train_mean.binaryproto'
    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open(MEAN_PROTO_PATH, 'rb' ).read()
    blob.ParseFromString(data)
    array = np.array(caffe.io.blobproto_to_array(blob))
    mean_npy = array[0]
    np.save(MEAN_NPY_PATH ,mean_npy)

    net = caffe.Net(deploy,caffe_model,caffe.TEST)
    transformer = caffe.io.Transformer({'data':net.blobs['data'].data.shape})
    transformer.set_mean('data', (np.load(MEAN_NPY_PATH).mean(1).mean(1))/255.)
    transformer.set_transpose('data', (2,0,1))
    transformer.set_channel_swap('data', (2,1,0))
    #transformer.set_raw_scale('data', 255.0)
    #net.blobs['data'].reshape(1,3,227,227)
    return net,transformer

def Test(img,net,transformer):
    im = caffe.io.load_image(img)
    net.blobs['data'].data[...] = transformer.preprocess('data', im)
    out = copy.deepcopy(net.forward())
    # out = net.forward_all(data=np.asarray([transformer.preprocess('data', im)]))
    return out['predict']

def process(predict,cols,rows):
    predict=predict.reshape((JOINTS,2))
    predict[:,0]+=0.5*cols
    predict[:,1]+=0.5*rows
    return predict

net,trans=Buildnet()

#---------------Read test set---------------#

testlist="./models/deeppose/test_images.txt"

predict=[]
label=[]
total_test=0

RATE=40

with open(testlist,'r') as f:
    for line in f.readlines():
        if random.randint(1,RATE)!=1:
            continue
        dat=(line.lstrip('\x00')).split()
        img=dat[0]
        label.append(np.zeros((JOINTS,2)))
        for i in range(0,JOINTS*2):
            label[-1][(i>>1),(i&1)]=eval(dat[i+1])
        label[-1]=process(label[-1],SIZE[0],SIZE[1])
        output=process(Test(img,net,trans),SIZE[0],SIZE[1])
        predict.append(output)
        total_test+=1
        if total_test%100==0:
            print "Read and Predict %d Images !"%total_test
    if total_test%100!=0:
        print "Read and Predict %d Images !"%total_test

predict=np.array(predict)
label=np.array(label)

#--------------------Accuracy----------------#

def dist(a,b):
    return np.sqrt(np.sum((a-b)**2))

def Diam(idx):
    return dist(label[idx,DIAM1],label[idx,DIAM2])

def PCPforOne(idx):
    detected=0;total=0
    lambd=0.5;
    for i in range(0,len(LIMBS)):
        x=LIMBS[i][0];y=LIMBS[i][1]
        leng=dist(label[idx,x],label[idx,y])
        total+=1;
        if dist(predict[idx,x],label[idx,x])<=leng*lambd and dist(predict[idx,y],label[idx,y])<=leng*lambd:
            detected+=1;
    return detected,total



def PDJforOne(idx,lambd):
    detected=0;total=0
    diam=Diam(idx);
    for i in range(0,JOINTS):
        total+=1
        if dist(label[idx,i],predict[idx,i])<=diam*lambd:
            detected+=1;
    return detected,total



def LossforOne(idx):
    ret=0
    for i in range(0,JOINTS):
        ret+=(dist(label[idx,i],predict[idx,i]))**2;
    return ret



def PCP():
    detected=0;total=0
    for i in range(0,total_test):
        ret1,ret2=PCPforOne(i)
        detected+=ret1
        total+=ret2
    return float(detected)/total



def PDJ(lambd):
    detected=0;total=0
    for i in range(0,total_test):
        ret1,ret2=PDJforOne(i,lambd)
        detected+=ret1
        total+=ret2
    return float(detected)/total;


def Loss():
    ret=0;
    for i in range(0,total_test):
        ret+=LossforOne(i)
    return ret/2.0/total_test/(JOINTS*2.0)

print "Loss: ",Loss()
print "PCP: ",PCP()
for i in range(5,80,5):
    lambd=i/100.0
    print "PDJ for norm %.2lf"%lambd,": %.6lf"%PDJ(lambd)

